In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', other_features=True)

data = Data('l', accent_classification=True)
data.generate_data('TEST_train',
                   'TEST_test',
                   'TEST_validate', force_override=False)

LOADING DATA...
LOAD SUCCESSFUL!


In [123]:
gen2 = data.generator('train', 16)
# test1 = next(gen1)
# test2 = next(gen2)

In [102]:
test2 = next(gen2)

In [129]:
test2 = next(gen2)
pos = 0
print(len(feature_dictionary))
print(np.array(test2[0][0]).shape)
print(np.array(test2[0][1]).shape)
print(len(test2[0]))
print(len(test2))
for el in test2[0][0]:
#     print(el)
    print (data.decode_x(el, dictionary))
#     print(data.decode_x_other_features(feature_dictionary, [el]))

# for el in test2[1]:
#     print(el)

BBB
10
(16, 23, 36)
(16, 150)
2
2
invitagen
iremirp
iksneko
jnavolimop
mišjentsačilevjan
mišjentsačilevjan
hišjenbordopjan
hišjenbordopjan
enitesjavd
amonobarab
enejlugo
amtsonlanoiseforp
anavokilboen
anavokilboen
ebžalot
invejark


In [6]:
def count_vowels(content, accented_vowels):
    num_all_vowels = 0
    for el in content:
        for m in list(el[3]):
            if m in accented_vowels:
                num_all_vowels += 1
    return num_all_vowels

count_vowels(content, accented_vowels)

592886

In [10]:
# print (X_train.shape)
# print (X_test.shape)
# print (X_validate.shape)
pos = 0
# print (decode_input(X_train[pos], dictionary))
# print (decode_X_features(feature_dictionary, [X_other_features_train[pos]]))
# print(decode_position(y_train[pos]))
# print('------------------------------------')
print (data.x_train.shape)
print (data.x_test.shape)
print (data.x_validate.shape)
# pos = 2
print (data.decode_x(data.x_train[pos], dictionary))
print (data.decode_x_other_features(feature_dictionary, [data.x_other_features_train[pos]]))
print(data.decode_y(data.y_train[pos]))
data.y_train[pos]

(430151, 23, 36)
(52058, 23, 36)
(54222, 23, 36)
ašjenlautkajan
Agsnpn-
Agsnpn-
[2, 5]


array([ 0.,  0.,  1.,  0.,  0.,  2.,  0.,  0.,  0.,  0.])

In [4]:
# num_examples = count_vowels(content, accented_vowels) # training set size
num_examples = 592886
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
actual_epoch = 1
num_fake_epoch = 20


# num_examples = len(data.x_train) # training set size
# nn_output_dim = 11
# nn_hdim = 516
# batch_size = 16
# actual_epoch = 1
# num_fake_epoch = 20



conv_input_shape=(23, 36)
othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(69, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(43, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [5]:
history = model.fit_generator(data.generator('train', batch_size), 
                              num_examples/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=num_examples/(batch_size * num_fake_epoch))

Epoch 1/20
1853/1852 [==============================] - 30s - loss: 0.2145 - actual_accuracy: 0.0880 - val_loss: 0.1650 - val_actual_accuracy: 0.2504
Epoch 2/20
1853/1852 [==============================] - 31s - loss: 0.1432 - actual_accuracy: 0.3922 - val_loss: 0.1132 - val_actual_accuracy: 0.5594
Epoch 3/20
1853/1852 [==============================] - 36s - loss: 0.1051 - actual_accuracy: 0.5976 - val_loss: 0.0803 - val_actual_accuracy: 0.7176
Epoch 4/20
1853/1852 [==============================] - 38s - loss: 0.0768 - actual_accuracy: 0.7317 - val_loss: 0.0575 - val_actual_accuracy: 0.8178
Epoch 5/20
1853/1852 [==============================] - 40s - loss: 0.0608 - actual_accuracy: 0.8016 - val_loss: 0.0447 - val_actual_accuracy: 0.8691
Epoch 6/20
1853/1852 [==============================] - 39s - loss: 0.0505 - actual_accuracy: 0.8397 - val_loss: 0.0399 - val_actual_accuracy: 0.8860
Epoch 7/20
1853/1852 [==============================] - 41s - loss: 0.0464 - actual_accuracy: 0.8566

In [6]:
name = '1_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [9]:
content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()